# Fraud Transaction Detection System

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [4]:
df = pd.read_csv('kenya_fraud_detection.csv')
df.head()

,Unnamed: 0,transaction_id,user_id,transaction_type,amount,location,device_type,network_provider,user_type,time_of_day,is_foreign_number,is_sim_recently_swapped,has_multiple_accounts,datetime
0,0,TX100000,user_8270,Withdraw Cash,2359.83,Kisumu,Android,Safaricom,individual,evening,0,0,0,2024-06-26 16:58:16
1,1,TX100001,user_1860,Lipa na M-Pesa,1042.99,Eldoret,iOS,Telkom Kenya,agent,morning,0,0,0,2024-06-10 04:15:22
2,2,TX100002,user_6390,Withdraw Cash,7567.65,Meru,Feature Phone,Safaricom,agent,afternoon,0,0,0,2024-06-02 23:48:44
3,3,TX100003,user_6191,Lipa na M-Pesa,3011.51,Nakuru,Android,Telkom Kenya,individual,evening,0,1,0,2024-06-06 00:18:47
4,4,TX100004,user_6734,Withdraw Cash,874.77,Nakuru,Feature Phone,Airtel,individual,night,0,0,0,2024-06-11 15:16:08


##